In [126]:
import cv2
import yaml
from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.functions import draw_landmarks
import matplotlib.pyplot as plt
import imageio
from utils.functions import cv_draw_landmark, get_suffix
import imageio_ffmpeg
import tqdm as tqdm
import numpy as np

In [89]:
video_fp_ = 'Dataset/Videos/000.mp4'

fn = video_fp_.split('/')[-1]
reader = imageio.get_reader(video_fp_)
fps = reader.get_meta_data()['fps']
suffix = get_suffix(video_fp_)
video_wfp = f'Dataset/Results{fn.replace(suffix, "")}_3d.mp4'
writer = imageio.get_writer(video_wfp, fps=fps)

In [67]:
# face detection
face_boxes = FaceBoxes()

In [ ]:
# regress 3DMM params
cfg = yaml.load(open('configs\mb1_120x120.yml'), Loader=yaml.SafeLoader)
gpu_mode = cfg.get('gpu_mode', False)
tddfa = TDDFA(gpu_mode = gpu_mode, **cfg)

In [132]:
dense_flag = True
pre_ver = None
boxes = []
boxes_b = []
param_lst_b = []
for i, frame in (enumerate(reader)):
    # print(frame)
    frame_bgr = frame[..., ::-1]  # RGB->BGR
    # print(frame_bgr)
    # if i == 0:
        # the first frame, detect face, here we only use the first face, you can change depending on your need
    boxes = face_boxes(frame_bgr)
    param_lst, roi_box_lst = tddfa(frame_bgr, boxes)
    boxes_b = boxes_b + boxes
    param_lst_b += param_lst
    np.savez_compressed('000.npz',boxes_b,param_lst_b)
